<a href="https://colab.research.google.com/github/Alan-Cheong/IEEE_QW_2020/blob/master/Crypto_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class Cryptocurrency:
    def __init__(self, max_supply, initial_reward, halving_interval):
        """
        Initialize the cryptocurrency parameters.
        :param max_supply: The maximum number of coins allowed.
        :param initial_reward: The initial reward per block mined.
        :param halving_interval: The number of blocks after which the reward is halved.
        """
        self.max_supply = max_supply
        self.current_supply = 0
        self.reward = initial_reward
        self.halving_interval = halving_interval
        self.blocks_mined = 0

    def mine_block(self):
        """
        Simulate mining a block and adjusting the reward if necessary.
        :return: The reward for mining the block, or 0 if max supply is reached.
        """
        if self.current_supply >= self.max_supply:
            print("Max supply reached. No more coins can be mined.")
            return 0

        # Add the current reward to the total supply
        coins_mined = min(self.reward, self.max_supply - self.current_supply)
        self.current_supply += coins_mined
        self.blocks_mined += 1

        # Adjust reward if halving interval is reached
        if self.blocks_mined % self.halving_interval == 0:
            self.reward /= 2

        return coins_mined

    def get_status(self):
        """
        Get the current status of the cryptocurrency.
        :return: A dictionary containing current stats.
        """
        return {
            "Current Supply": self.current_supply,
            "Reward per Block": self.reward,
            "Blocks Mined": self.blocks_mined,
            "Remaining Supply": self.max_supply - self.current_supply,
        }


# Example usage
if __name__ == "__main__":
    # Define cryptocurrency parameters
    MAX_SUPPLY = 21_000_000  # Maximum coin supply
    INITIAL_REWARD = 50      # Initial block reward
    HALVING_INTERVAL = 210_000  # Blocks per halving (Bitcoin uses this value)

    # Create cryptocurrency
    crypto = Cryptocurrency(MAX_SUPPLY, INITIAL_REWARD, HALVING_INTERVAL)

    # Simulate mining
    for _ in range(10):  # Simulate 10 blocks being mined
        reward = crypto.mine_block()
        print(f"Block {_ + 1} mined! Reward: {reward} coins")

    # Display final status
    print("\nFinal Cryptocurrency Status:")
    print(crypto.get_status())


Block 1 mined! Reward: 50 coins
Block 2 mined! Reward: 50 coins
Block 3 mined! Reward: 50 coins
Block 4 mined! Reward: 50 coins
Block 5 mined! Reward: 50 coins
Block 6 mined! Reward: 50 coins
Block 7 mined! Reward: 50 coins
Block 8 mined! Reward: 50 coins
Block 9 mined! Reward: 50 coins
Block 10 mined! Reward: 50 coins

Final Cryptocurrency Status:
{'Current Supply': 500, 'Reward per Block': 50, 'Blocks Mined': 10, 'Remaining Supply': 20999500}
